In [ ]:
%load_ext autoreload
%autoreload 2

# Get parent directory and add to sys.path
import os
import sys

parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

# Require ipympl
%matplotlib widget 

In [ ]:
# MPC import
import matplotlib.pyplot as plt
import numpy as np
from LinearMPC.MPCVelControl import MPCVelControl
from src.rocket import Rocket
from src.vel_rocket_vis import RocketVis, plot_static_states_inputs
from LinearMPC.utils import GAMA, VX, VY, VZ

rocket_obj_path = os.path.join(parent_dir, "Cartoon_rocket.obj")
rocket_params_path = os.path.join(parent_dir, "rocket.yaml")

In [ ]:
Ts = 0.05
sim_time = 10
H = 10

# Since the system is divided in independent subsystems, 
# we can put the initial state as follows which is equivalent to checking each subsystem separately
x0 = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=float)
x0[VX] = 5
x0[VY] = 5
x0[VZ] = 5
x0[GAMA] = np.deg2rad(40)

rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)

xs, us = rocket.trim()
A, B = rocket.linearize(xs, us)
mpc = MPCVelControl().new_controller(rocket, Ts, H)

t_cl, x_cl, u_cl, t_ol, x_ol, u_ol, _ = rocket.simulate_control(
    mpc, sim_time, H, x0, method="linear")

In [ ]:
plot_static_states_inputs(t_cl[:-1], x_cl[:,:-1], u_cl)
plot_static_states_inputs(t_ol[:-1, 0], x_ol[:, :-1,0], u_ol[:, :, 0])

In [ ]:
vis = RocketVis(rocket, rocket_obj_path)
vis.anim_rate = 0.7
vis.animate(t_cl[:-1], x_cl[:, :-1], u_cl, T_ol=t_ol[:, :-1], X_ol=x_ol, U_ol=u_ol);

In [ ]:
mpc.mpc_x.plot_max_invariant_set()
mpc.mpc_y.plot_max_invariant_set()
mpc.mpc_z.plot_max_invariant_set()
mpc.mpc_roll.plot_max_invariant_set()